In [ ]:
# Parameters
day_obs = '2024-06-26'
instruments = 'latiss, lsstcomcamsim, lsstcomcam'

In [ ]:
import os
import time as t
import pandas as pd
import matplotlib.pylab as plt
from IPython.display import display, Markdown, display_markdown

In [ ]:
### 'Get Consdb access'
from lsst.summit.utils import ConsDbClient

URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one
os.environ["no_proxy"] += ",.consdb"

client = ConsDbClient(URL)
display_markdown('### Consolidated Database is accessible',raw=True)

In [ ]:
# Put Plot in backend
plt.style.use('seaborn-v0_8-bright')
def plot(y, x):
    # plot
    fig, ax = plt.subplots()
    ax.scatter(x, y)#, linewidth=2.0, color="#058b8c")

    plt.show()


In [ ]:
day_obs_int = int(day_obs.replace('-', ''))
instrument_list = [ins.strip() for ins in instruments.split(',')]

for instrument in instrument_list:
    print("------------------------------------------------------------")
    print()
    display_markdown(f'# Instrument: {instrument}',raw=True)
    #################### Put in Backend
    ccdvisit1_quicklook = f'''
        SELECT * FROM cdb_{instrument}.ccdvisit1_quicklook
    '''

    visit1 = f'''
        SELECT * FROM cdb_{instrument}.visit1
    '''

    # Join Visit1 and ccdVisit1 to access data and day obs
    visits = client.query(visit1).to_pandas()
    quicklook = client.query(ccdvisit1_quicklook).to_pandas()

    visits = visits.join(quicklook, on='visit_id',lsuffix='',rsuffix='_q')
    #################### Put in Backend - end

    # If we see data exist in psf, zero, then we should pare down like visits_today below
    try:
        visits_w_psf = visits[visits['psf_area'].notna()]
        time = visits_w_psf['obs_start']
        display_markdown(f'Number of visits with psf_area populated {len(visits_w_psf)}', raw=True)
        display_markdown('## psf_area vs obs_start', raw=True)
        plot(time, visits_w_psf['psf_area'])
    except KeyError as err:
        display_markdown(f"Psf_area not a column in {instrument} dataframe",raw=True)
        display_markdown(f"key error for {err}", raw=True)

    try:
        visits_w_zero = visits[visits['zero_point'].notna()]
        time = visits_w_zero['obs_start']
        display_markdown(f'Number of visits with zero_point populated {len(visits_w_zero)}', raw=True)
        display_markdown('## zero_point vs obs_start', raw=True)
        plot(time, visits_w_zero['zero_point'])
    except KeyError as err:
        display_markdown(f"Zero_point not a column in {instrument} dataframe", raw=True)
        display_markdown(f"key error for {err}", raw=True)

    # Pare down to only day obs
    visits_today = visits[(visits['day_obs'] == day_obs_int)]
    display_markdown(f"How many visits today? {len(visits_today)}", raw=True)

    ra = visits_today['s_ra']
    dec = visits_today['s_dec']        
    display_markdown(f"How many ra? {len(ra)}", raw=True)
    display_markdown(f"How many dec? {len(dec)}", raw=True)

    display_markdown(f'Ra Dec should be populated for {instrument}', raw=True)
    plot(ra, dec)
    t.sleep(.5)